<a href="https://colab.research.google.com/github/PilarAriasVazquez/AA_PRACTICA2_GRUPO_7/blob/main/Entrega2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
pip install plotly_express

las predicciones de amount ponemos 0

In [8]:

import pandas as pd
import plotly_express as px
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

In [9]:
df = pd.read_csv('volume_us_forecast_sample.csv', sep=',')
print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1831293 entries, 0 to 1831292
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   COUNTRY                  object 
 1   GEOGRAPHY                object 
 2   CUSTOMER                 object 
 3   SEGMENT/CHANNEL          object 
 4   PRODUCT                  object 
 5   YEAR                     int64  
 6   WEEK                     int64  
 7   SCENARIO                 object 
 8   SCENARIO_LVL_2           object 
 9   AMOUNT                   float64
 10  MODEL_ID                 float64
 11  MODEL_CAT                object 
 12  DEFAULT_MODEL            int64  
 13  FORECAST_YEAR            float64
 14  FORECAST                 object 
 15  UNIT                     object 
 16  AMOUNT_WITHOUT_OUTLIERS  float64
dtypes: float64(4), int64(3), object(10)
memory usage: 237.5+ MB
None


,COUNTRY,GEOGRAPHY,CUSTOMER,SEGMENT/CHANNEL,PRODUCT,YEAR,WEEK,SCENARIO,SCENARIO_LVL_2,AMOUNT,MODEL_ID,MODEL_CAT,DEFAULT_MODEL,FORECAST_YEAR,FORECAST,UNIT,AMOUNT_WITHOUT_OUTLIERS
0,United States,West,AO_FTN_US,CG3_Z59,L2488,2023,20,AI_forecast,Volume,-5.084845,1.0,Prophet,1,2023.0,W12F,Gallons,NaN
1,United States,West,AO_FTN_US,CG3_Z59,L2488,2023,21,AI_forecast,Volume,9.696815,1.0,Prophet,1,2023.0,W12F,Gallons,NaN
2,United States,West,AO_FTN_US,CG3_Z59,L2488,2023,22,AI_forecast,Volume,-5.896414,1.0,Prophet,1,2023.0,W12F,Gallons,NaN
3,United States,West,AO_FTN_US,CG3_Z59,L2488,2023,23,AI_forecast,Volume,0.544680,1.0,Prophet,1,2023.0,W12F,Gallons,NaN
4,United States,West,AO_FTN_US,CG3_Z59,L2488,2023,24,AI_forecast,Volume,8.710286,1.0,Prophet,1,2023.0,W12F,Gallons,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1831288,United States,Central,1000007206,CG2_Z27,L2399,2023,11,AI_forecast,Volume,0.003348,0.0,Prophet,0,2023.0,W07F,Gallons,NaN
1831289,United States,Central,1000007206,CG2_Z27,L2399,2023,12,AI_forecast,Volume,0.005433,0.0,Prophet,0,2023.0,W07F,Gallons,NaN
1831290,United States,Central,1000007206,CG2_Z27,L2399,2023,13,AI_forecast,Volume,0.007053,0.0,Prophet,0,2023.0,W07F,Gallons,NaN
1831291,United States,Central,1000007206,CG2_Z27,L2399,2023,14,AI_forecast,Volume,0.005785,0.0,Prophet,0,2023.0,W07F,Gallons,NaN


- 5 combinaciones más grandes de canal - producto por customer (actuals)
- Error cuadrático de los canales - productos más grandes de cada customer (de cada model ID )
- Previsioens de ejercicio 2 de cada modeID

In [17]:
df['SCENARIO'].value_counts()

AI_forecast    1467072
actual          364221
Name: SCENARIO, dtype: int64

In [36]:
df_actuals = (df
              .loc[df['SCENARIO']=='actual']
              .groupby(['CUSTOMER', 'SEGMENT/CHANNEL','PRODUCT'])['AMOUNT'].sum()
              .reset_index()
              .sort_values(by='AMOUNT', ascending = False)
              )

In [38]:
top_combinations = df_actuals.groupby('CUSTOMER').head(5)

print(top_combinations)

        CUSTOMER SEGMENT/CHANNEL PRODUCT        AMOUNT
233   1000006156         CG2_Z19   L2296  2.411632e+08
212   1000006156         CG2_Z19   L2254  2.243821e+08
341   1000007013         CG2_Z30   L2254  2.212933e+08
876    AO_FTN_US             2FS   L2481  2.186362e+08
350   1000007013         CG2_Z30   L2300  1.595024e+08
1002   AO_FTN_US   ZZ_FSV_FTN_US   L2481  1.416175e+08
239   1000006156         CG2_Z19   L2306  1.116108e+08
228   1000006156         CG2_Z19   L2291  8.861778e+07
248   1000006156         CG2_Z19   L2315  6.559889e+07
912    AO_FTN_US         CG3_Z24   L2481  5.868940e+07
370   1000007013         CG2_Z30   L2381  4.276457e+07
449   1000007206         CG2_Z01   L2350  3.769028e+07
947    AO_FTN_US         CG3_Z49   L2481  2.815524e+07
647   1000008742         CG2_Z08   L2253  2.649091e+07
670   1000008742         CG2_Z08   L2306  2.574832e+07
436   1000007206         CG2_Z01   L2311  2.306756e+07
995    AO_FTN_US         CG3_Z81   L2481  2.239784e+07
661   1000

In [ ]:
# Ordenar por la suma del 'AMOUNT' en orden descendente
grouped = grouped.sort_values(by='AMOUNT', ascending=False)

# Obtener las 5 combinaciones más grandes para cada cliente
top_combinations = grouped.groupby('CUSTOMER').head(5)

print(top_combinations)

In [31]:
valores_unicos = df_actuals['CUSTOMER'].unique()
print(valores_unicos)


['1000006156' '1000007013' '1000007206' '1000008742' 'AO_FTN_US']
